In [1]:
#!/bin/python3

import math
import os
import random
import re
import sys

# Complete the maxCircle function below.


def dfs(graph, vertex):
    stack = []
    path = []
    
    stack.append(vertex)
    
    while(stack):
        cur = stack.pop()
        path.append(cur)
        
        unvisited_neighbors = [x for x in graph[cur] if x not in path and x not in stack ]
        stack = stack + unvisited_neighbors
        
    return(path) 

def maxCircle_naive_search(queries):
    # creat graph
    graph = dict()
    output = []
    for q in queries:
        # we need to create dict in both directions, i.e., an undirected graph
        if q[0] not in graph:
            graph[q[0]] = []
        if q[1] not in graph:
            graph[q[1]] = []
        
        if q[1] not in graph[q[0]]:
            graph[q[0]].append(q[1])
        if q[0] not in graph[q[1]]:
            graph[q[1]].append(q[0])
        
        # Now graph stores the graph from the first q queries, doing search say using dfs 
        unique_node = list(graph.keys())
        visited = []
        running_max = 0
        # loop through all nodes in the graph, group has minimial size of 2
        for i in unique_node:
            # only calculate dfs when i has not been visited
            if i not in visited:
                this_group = dfs(graph, i)
                visited = visited + this_group
                if len(this_group) > running_max:
                    running_max = len(this_group)
        
        output.append(running_max)
        
    return(output)

def maxCircle_naive_search2(queries):
    # Naive search1 timed out for too many cases
    
    # creat graph
    graph = dict()
    output = []
    running_max = 0
    for q in queries:
        # we need to create dict in both directions, i.e., an undirected graph
        if q[0] not in graph:
            graph[q[0]] = []
        if q[1] not in graph:
            graph[q[1]] = []
        
        if q[1] not in graph[q[0]]:
            graph[q[0]].append(q[1])
        if q[0] not in graph[q[1]]:
            graph[q[1]].append(q[0])
        
        # Now graph stores the graph from the first q queries, doing search say using dfs 
       
        visited = []        
        # Do NOT loop through all nodes in the graph, group has minimial size of 2
        # only calculate dfs for nodes that are involved in the newly added query
        for i in q:            
            if i not in visited:
                this_group = dfs(graph, i)
                visited = visited + this_group
                if len(this_group) > running_max:
                    running_max = len(this_group)
        
        output.append(running_max)
    
    # note that this is still not perfect since if there are n queries, it goes like O(n^2) since for each query worst case is linear time 
    return(output)

class UnionFind():
    def __init__(self, N):
        # N is the maximum number of items. 
        self.size = [1 for x in range(N)]
        self.parent_arr = list(range(N))
    def find_root(self, i):
        while (self.parent_arr[i]!=i):
            i = self.parent_arr[i]
        return(i)
    def connected(self, a,b):
        root_a = self.find_root(a)
        root_b = self.find_root(b)
        return(root_a==root_b)
    def union(self, a, b):
        root_a = self.find_root(a)
        root_b = self.find_root(b)
        # only when a,b are not connected, otherwise, the second branch can accidentally double the sizes when a,b are connected
        if (root_a != root_b):
            if self.size[root_a] < self.size[root_b]:
                self.parent_arr[root_a] = root_b
                self.size[root_b] += self.size[root_a]
            else:
                self.parent_arr[root_b] = root_a
                self.size[root_a] += self.size[root_b]


def maxCircle_unionfind(queries):
    # Naive search1 and search2 both timed out for some test cases
    # We use balanced unionfind 
    
    # create dict for all nodes involved
    nodes = []
    for q in queries:
        nodes += q
    unique_nodes = list(set(nodes))    
    nodes_dict = {x:i for i,x in enumerate(unique_nodes)}
            
    # creat Unionfind
    uf = UnionFind(len(unique_nodes))
    output = []
    running_max = 0
    for q in queries:
        # union
        uf.union(nodes_dict[q[0]],nodes_dict[q[1]])
        # return largest circle: the only possible condidates are the root of the newly added query
        candidate = uf.size[uf.find_root(nodes_dict[q[0]])] # q[0] or q[1] does NOT matter since they belong to 1 connected component for sure
        if running_max < candidate:
            running_max = candidate
        
        # O(Mlog(N)) M being the number queries and N being the number of nodes
       
        output.append(running_max)

    return(output)

def maxCircle(queries):
    return(maxCircle_unionfind(queries))